## Etape 1 : Scraping des résultats depuis bases.athle

Objectifs :
+ créer un pipeline pour récupérer les données d'une compétition d'athlétisme (course sur route, trail, cross, ...)
+ retourner ces données sous la forme d'une table relationnelle
+ par la suite, exploiter cette table pour créer des statistiques et visualisations sur les compétitions

Format d'une URL de bases.athle :
+ ? avant la requête
+ & pour ajouter des paramètres à la recherche 

bases.athle.fr/asp.net/liste.aspx**?frmbase=resultats**&**frmmode=1**&**frmespace=0**&**frmcompetition=**274211

Liste des paramètres pour une recherche par athlète :
+ frmsaison
+ frmclub
+ frmnom
+ frmprenom
+ frmsexe
+ frmlicence
+ frmdepartement
+ frmligue
+ frmcomprch


### Installation de la libraire lxml

In [1]:
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.9 MB/s eta 0:00:0000:0100:01


### Import des librairies requises

In [2]:
import lxml
import urllib
import bs4
import pandas as pd
import re
from urllib import request

In [3]:
#Exemple de résultats d'une compétition
url="https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=253657"

In [4]:
request_text = request.urlopen(url).read()

In [5]:
page = bs4.BeautifulSoup(request_text, "lxml")

### Dictionnaires utiles

#### Catégories de performance et cotation d'après table FFA

In [6]:
#bareme et table de cotation
#source: https://www.athle.fr/asp.net/main.html/html.aspx?htmlid=125

perf = {'IA': 40, 'IB': 35, #international
        'N1': 30, 'N2': 28, 'N3': 26, 'N4': 24, #national
        'IR1': 21, 'IR2': 20, 'IR3': 19, 'IR4': 18, #inter-regional
        'R1': 15, 'R2': 14, 'R3': 13, 'R4': 12, 'R5': 11, 'R6': 10, #regional
        'D1': 8, 'D2': 7, 'D3': 6, 'D4': 5, 'D5': 4, 'D6': 3, 'D7': 2} #departemental

#### Catégories d'âge d'après la réglementation FFA

In [7]:
#categories d'âge en 2023
#source: https://www.athle.fr/asp.net/main.html/html.aspx?htmlid=25

cat = {'M10': 'Masters 10', 'M9': 'Masters 9', 'M8': 'Masters 8', 'M7': 'Masters 7', 'M6': 'Masters 6',
       'M5': 'Masters 5', 'M4': 'Masters 4', 'M3': 'Masters 3', 'M2': 'Masters 2', 'M1': 'Masters 1', 'M0': 'Masters 0',
       'SE': 'Seniors',
       'JU': 'Juniors',
       'CA': 'Cadet.te.s',
       'MI': 'Minimes',
       'BE': 'Benjamin.e.s',
       'PO': 'Poussins',
       'EA': 'École d\'Athlétisme',
       'BB': 'Baby Athlé'}

## Date, nom, lieu, ligue, label et département de la compétition

In [8]:
header = page.find('div', {'class' : "mainheaders"})
header=str(header)
print(header)

<div class="mainheaders">06/11/22 - 10 km de Riorges<br/><span style="font-size:13px">Riorges - ARA - 042 - Label Régional</span></div>


#### Nom de la compétition

In [9]:
re_nom = re.compile("(?<=\- )(.*?)(?=\<)")
text = re_nom.findall(header)
nom = text[0]
sous_titre = text[1]
print(nom)

10 km de Riorges


#### Lieu de la compétition

In [10]:
re_lieu = re.compile("(?<=\>)(\D*?)(?=\ -)")
lieu = re_lieu.findall(header)[0]
print(lieu)

Riorges


#### Date de la compétition

In [11]:
re_date = re.compile("[0-9]{2}/[0-9]{2}/[0-9]{2}")
date = re_date.findall(header)[0]
print(date)

06/11/22


#### Ligue de la compétition

In [12]:
soustitre = nom[1]
re_ligue = re.compile("[A-Z]{3}")
ligue = re_ligue.findall(sous_titre)[0]
print(ligue)

ARA


#### Département de la compétition

In [13]:
re_dept = re.compile("[0-9]{3}")
dept = re_dept.findall(sous_titre)[0]
print(dept)

042


#### Label de la compétition (en cours)

In [14]:
re_label = re.compile("(?<=Label ).*(?<!')")
try:
    label = re_label.findall(sous_titre)[0]
    print(label)
except IndexError:
    print("Pas de label pour cette compétition")

Régional


#### Données de la compétition sous forme de dictionnaire

In [15]:
compet= {'nom': nom,
      'lieu': lieu,
      'date': date,
      'ligue': ligue,
      'dept': dept,
      'label': label}

In [16]:
compet['dept']

'042'

In [17]:
print(page.find("title"))

<title>les Résultats des Compétitions</title>


In [18]:
print(page.find("mainheaders"))

None


In [19]:
print("Il y a", len(page.findAll("tr")), "éléments dans la page qui sont des <tr>")

Il y a 257 éléments dans la page qui sont des <tr>


In [20]:
print(page.find('a', {'title' : 'Records principaux pour cette épreuve'}))

<a class="white" href="/asp.net/liste.aspx?frmbase=records&amp;frmmode=2&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=261&amp;frmsexe=M&amp;frmserie=10 Km Route TC" title="Records principaux pour cette épreuve">10 Km Route TC</a>


In [21]:
print(page.find('tr'))

<tr> <td style="width:2px"><div class="separator"></div></td> <td class="barButtons"><span class="barButtonsOff" onclick="history.back()" onmouseout="this.className='barButtonsOff'" onmouseover="this.className='barButtonsOn'" style="width:25px;background-image:url('/images/v3/bar.back.gif')" title="Retour"> </span></td> <td style="width:2px"><div class="separator"></div></td><td class="barInputs"> <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmposition=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Page &gt; 001/002 &lt;</option><option value="1">Page &gt; 002/002 &lt;</option></select>  <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmtri=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Tri &gt; Cat./Sex./

## Athlètes et chronos de la compétition

In [22]:
text = str(page.findAll('td', {'class' : 'datas1'}))
text = text.replace("<td class=\"datas1\">", "")
text = text.replace("</b>", "")
text = text.replace("<b>", "")
text = text.replace("<td>", "")
text = text.replace("</td>", "")

In [36]:
page.findAll('td', {'class' : 'datas1'})[6]

<td class="datas1"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=10 Km Route TC&amp;frmcategorie=SE&amp;frmsexe=M" title="Résultats pour la catégorie du participant">SEM</a>/89</td>

In [120]:
headers = []
for i in page.find_all('td'):
    title = i.text
    headers.append(title)

In [142]:
L=[]
for i in page.find_all('tr'):
    L.append(i)

### Chronos de la compétition

In [151]:
temps=[]
for i in L:
    temps.append(re_temps.findall(str(i)))
temps=[x for x in temps if x!=[]]
for i in range(len(temps)):
    temps[i]=temps[i][0].replace("<b>","")
temps

["30'58''",
 "31'30''",
 "32'01''",
 "32'15''",
 "32'40''",
 "33'02''",
 "33'38''",
 "33'41''",
 "34'54''",
 "35'16''",
 "35'35''",
 "35'36''",
 "35'39''",
 "35'40''",
 "35'42''",
 "35'42''",
 "35'44''",
 "35'50''",
 "35'53''",
 "35'56''",
 "36'04''",
 "36'05''",
 "36'21''",
 "36'28''",
 "36'39''",
 "36'41''",
 "36'46''",
 "36'46''",
 "36'50''",
 "36'52''",
 "36'55''",
 "36'58''",
 "37'05''",
 "37'11''",
 "37'13''",
 "37'22''",
 "37'27''",
 "37'32''",
 "37'37''",
 "37'37''",
 "37'41''",
 "37'42''",
 "37'50''",
 "37'54''",
 "37'59''",
 "38'00''",
 "38'05''",
 "38'10''",
 "38'11''",
 "38'15''",
 "38'26''",
 "38'26''",
 "38'27''",
 "38'35''",
 "38'41''",
 "38'49''",
 "38'54''",
 "38'56''",
 "39'09''",
 "39'16''",
 "39'17''",
 "39'32''",
 "39'35''",
 "39'55''",
 "40'02''",
 "40'04''",
 "40'04''",
 "40'04''",
 "40'14''",
 "40'15''",
 "40'41''",
 "40'46''",
 "40'52''",
 "41'00''",
 "41'08''",
 "41'18''",
 "41'21''",
 "41'25''",
 "41'30''",
 "41'42''",
 "41'43''",
 "41'47''",
 "41'53''",
 "42

30'58''
31'30''
32'01''
32'15''
32'40''
33'02''
33'38''
33'41''
34'54''
35'16''
35'35''
35'36''
35'39''
35'40''
35'42''
35'42''
35'44''
35'50''
35'53''
35'56''
36'04''
36'05''
36'21''
36'28''
36'39''
36'41''
36'46''
36'46''
36'50''
36'52''
36'55''
36'58''
37'05''
37'11''
37'13''
37'22''
37'27''
37'32''
37'37''
37'37''
37'41''
37'42''
37'50''
37'54''
37'59''
38'00''
38'05''
38'10''
38'11''
38'15''
38'26''
38'26''
38'27''
38'35''
38'41''
38'49''
38'54''
38'56''
39'09''
39'16''
39'17''
39'32''
39'35''
39'55''
40'02''
40'04''
40'04''
40'04''
40'14''
40'15''
40'41''
40'46''
40'52''
41'00''
41'08''
41'18''
41'21''
41'25''
41'30''
41'42''
41'43''
41'47''
41'53''
42'01''
42'07''
42'08''
42'22''
42'28''
42'29''
42'39''
42'41''
42'44''
42'46''
42'51''
42'52''
43'03''
43'03''
43'06''
43'10''
43'16''
43'17''
43'18''
43'26''
43'40''
43'41''
43'45''
43'55''
44'05''
44'08''
44'09''
44'15''
44'34''
44'43''
44'56''
44'59''
45'11''
45'24''
45'25''
45'37''
45'46''
45'51''
45'54''
46'02''
46'09''
46'09''


In [74]:
re_temps = re.compile("<b>(?=\d).*(?=<\/b>)")

In [87]:
string=str(L).replace("\","")

SyntaxError: unterminated string literal (detected at line 1) (229407386.py, line 1)

In [88]:
re_temps.findall(str(L))

['<b>30\'58\'\'</b></td><td class="separator3"></td><td class="datas0"><a href="javascript:bddThrowAthlete(\'resultats\', 43141, 0)">LORTON Guillaume</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;pardisplay=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmclub=003017">Montlucon Athletisme</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;FrmDepartement=003">003</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;FrmLigue=ARA">ARA</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=10 Km Route TC&amp;frmcategorie=SE&amp;frmsexe=M" title="Résultats pour la catégorie du partici

In [63]:
re.temps = re.compile("(?<=<b>).*(?<!</b>)")
L[6]

<tr><td class="datas0">1</td><td class="separator3"></td><td class="datas0"><b>30'58''</b></td><td class="separator3"></td><td class="datas0"><a href="javascript:bddThrowAthlete('resultats', 43141, 0)">LORTON Guillaume</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;pardisplay=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmclub=003017">Montlucon Athletisme</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;FrmDepartement=003">003</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;FrmLigue=ARA">ARA</a></td><td class="separator3"></td><td class="datas0"><a href="/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=10 Km Route TC&amp;frmcate